# ***Test questions***

In [ ]:
!pip -q install transformers peft bitsandbytes datasets accelerate trl



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 532.9/532.9 kB 25.6 MB/s eta 0:00:00


In [ ]:
import torch
print("GPU:", torch.cuda.get_device_name(0))


GPU: Tesla T4


In [ ]:
import os
print(os.listdir())


['.config', 'test_prompts.txt', '.ipynb_checkpoints', 'train.jsonl']


load json properly

In [ ]:
!wc -l train.jsonl
import json

data = []

with open("train.jsonl", "r", encoding="utf-8") as f:
    for line in f:
        data.append(json.loads(line))

print("Total samples:", len(data))
print("\nFirst sample:\n", data[0])



150 train.jsonl
Total samples: 150

First sample:
 {'instruction': 'What is gradient descent?', 'response': '• Minimizes loss function\n• Updates weights iteratively\n• Moves opposite gradient direction\n• Stops at convergence'}


checking consistency


In [ ]:
for i in range(3):
    print("\nQ:", data[i]["instruction"])
    print(data[i]["response"])



Q: What is gradient descent?
• Minimizes loss function
• Updates weights iteratively
• Moves opposite gradient direction
• Stops at convergence

Q: Explain overfitting.
• Model memorizes training data
• Performs poorly on unseen data
• High variance problem
• Caused by excessive complexity

Q: What is regularization?
• Technique to prevent overfitting
• Adds penalty to loss function
• Reduces model complexity
• Improves generalization


In [1]:
!pip -q install transformers accelerate


# **load base model**

In [2]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

model_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"

tokenizer = AutoTokenizer.from_pretrained(model_name)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    device_map="auto"
)

generator = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer
)

print("Model loaded")


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

...

`torch_dtype` is deprecated! Use `dtype` instead!


model.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

Device set to use cuda:0


Model loaded


# **Load your test prompts**

In [4]:
with open("test_prompts.txt") as f:
    test_prompts = [line.strip() for line in f.readlines()]

len(test_prompts)


20

# **Run baseline inference**

In [5]:
baseline_outputs = []

for q in test_prompts:
    prompt = f"Answer briefly:\n{q}\n"

    out = generator(
        prompt,
        max_new_tokens=80,
        do_sample=False
    )[0]["generated_text"]

    answer = out.replace(prompt, "").strip()

    baseline_outputs.append((q, answer))

print(baseline_outputs[0])


The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


('Explain gradient descent.', '2. Gradient Descent Algorithm:\nGradient descent is a popular algorithm used for solving linear regression problems. It is a simple and efficient algorithm that works by iteratively updating the weights of the neural network.\n\n3. Gradient Descent with Momentum:\nGradient descent with momentum is an extension of gradient descent that incorporates momentum to improve the convergence rate. It')


In [6]:
for i in range(3):
    print("\n---")
    print("Q:", baseline_outputs[i][0])
    print("A:", baseline_outputs[i][1])



---
Q: Explain gradient descent.
A: 2. Gradient Descent Algorithm:
Gradient descent is a popular algorithm used for solving linear regression problems. It is a simple and efficient algorithm that works by iteratively updating the weights of the neural network.

3. Gradient Descent with Momentum:
Gradient descent with momentum is an extension of gradient descent that incorporates momentum to improve the convergence rate. It

---
Q: What is overfitting in machine learning?
A: 2. Explain the concept of underfitting in machine learning

Overfitting occurs when the model is trained on a small portion of the data, resulting in a model that performs well on the training data but not on the test data. Underfitting occurs when the model is trained on a large portion of the data, resulting in a model that performs well on the training data but

---
Q: Difference between bagging and boosting?
A: 1. Bagging: Bagging is a technique used in supervised learning where the training data is divided int

save the o/p to a file

In [7]:
with open("baseline_outputs.txt", "w", encoding="utf-8") as f:
    for q, a in baseline_outputs:
        f.write(f"Q: {q}\n{a}\n\n{'='*40}\n\n")

print("Saved baseline outputs")


Saved baseline outputs
